In [1]:
"""" Introduction 

[JP]
このプログラムは欠損データセットをi)平均値補完 ii)k-NN法 iii)ランダムフォレストiv) XGBoostを使って
補完した疑似完全データセットを用い、RNNによりPV発電出力を予測するものです。

[ENG] 
This program is made for the prediction of PV output.
In this program,dataset imputed by 
  i)Mean Imputation 
  ii)k-NN Method 
  iii)Random Forest 
  iv)XGBoost
is used for input data.
""""



####### ライブラリのインポート ######
import tensorflow as tf
import numpy as np
import pandas as pd
import math as ma
import matplotlib.pyplot as plt
import os
import shutil

from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
tf.get_logger().setLevel("ERROR")


###### グラフ作成 ######
def graph(time_stamp, pred, target, Lower, Upper, name, graph_number):
    Figure = plt.figure()
    graph_legend  = "Predict(" + name + ")"
    plt.plot(time_stamp, pred, "blue")
    plt.plot(time_stamp, target, "red")
    if graph_number == 3:
        plt.plot(time_stamp,Lower, "green")
        plt.plot(time_stamp,Upper, "green")
        plt.legend(["Predict", "Target", "Prediction Interval"]) # 凡例
    else:
        plt.legend([graph_legend, "target"]) # 凡例
    plt.xlabel("time [hour]") # 横軸
    plt.ylabel("generation [kW]") # 縦軸
    plt.close()
    
    return Figure
      
###### RMSE ######
def RMSE(pred, Predict, name):
    new_name = "Score(" + name + "): %.2f RMSE"
    testScore = ma.sqrt(mean_squared_error(pred[:,0], Predict[:,0]))
    return testScore

###### ファイル保存 ######
def save_file_at_dir(filename, new_path, mode='w'):
    os.makedirs(new_path, exist_ok=True)
    shutil.move(f'{filename}', f'./{new_path}')

In [4]:
############################## データを読み込む & 学習・テストデータに分割する #################################
        
for mr in range(4):
    for m in range (4):
        ###### 欠損率・補完手法を指定し、データセットを読み込む
        method = ['Mean','KNN','RF','XGB']
        missing_rate = ['20','40','60','80']
        path_name = f"{method[m]}{missing_rate[mr]}%"
        PV_data = pd.read_csv(f"MCAR{missing_rate[mr]}%_{method[m]}.csv")

        ###### 読み込んだデータに月、時刻の三角関数成分を加える
        sin_month = np.sin(PV_data["month"]/6*(ma.pi))
        cos_month = np.sin(PV_data["month"]/6*(ma.pi))
        sin_hour = np.sin(PV_data["hour"]/12*(ma.pi)) 
        cos_hour = np.cos(PV_data["hour"]/12*(ma.pi)) 
        time_data = pd.concat([sin_month,cos_month,sin_hour,cos_hour], axis=1) 
        name = ["sin_month","cos_month","sin_hour", "cos_hour"] 
        time_data.columns = name         
        PV_data = pd.concat([PV_data, time_data], axis=1) 

        ###### 学習データ・テストデータ・予測データの日数を指定
        train_days = 255 
        test_days = 31 
        pred_days = 31
        time_stamps = PV_data["hour"]
        time_stamp = time_stamps[:48]
        row = len(PV_data) 
        col = len(PV_data.columns)

        ###### 具体的な学習データ・テストデータの日を指定
        train = PV_data[:48*train_days] 
        test = PV_data[48*train_days:48*(train_days + test_days)] 

        X_train = train[["sin_month","cos_month","sin_hour", "cos_hour","humidity","temp", "cloudcover", "rain"]] 
        y_train = train["generation"] 

        X_test = test[["sin_month","cos_month","sin_hour", "cos_hour","humidity","temp", "cloudcover", "rain"]] 
        y_test = test["generation"] 
        
        X_train = (X_train.values)
        y_train = (y_train.values)
        X_test = (X_test.values)
        y_test = (y_test.values) 

        X_train = X_train.reshape((train_days,48, 8)) # 3次元に変換
        y_train = y_train.reshape((train_days,48, 1)) # 3次元に変換
        X_test = X_test.reshape((test_days,48, 8)) # 3次元に変換
        y_test = y_test.reshape((test_days,48, 1)) # 3次元に変換
        
########################################### RNNの構築と学習 #################################################  
        ###### モデルの構築 
        input_dim = 8 # 入力データの要素数
        output_dim = 1 # 出力データ数
        NN_hidden_units_1 = 30 # 隠れ層（第一層）のユニット数
        NN_hidden_units = 20 # 隠れ層（第二層）のユニット数
        len_sequence = 48 # 時系列の長さ
        batch_size = 128 # ミニパッチサイズ
        num_of_training_epochs = 300
        learning_rate = 0.01 # 学習率

        NN_model = Sequential()
        NN_model.add(SimpleRNN(NN_hidden_units_1, input_shape=(len_sequence, input_dim), activation = "relu", return_sequences=True)) # 1層目
        NN_model.add(SimpleRNN(NN_hidden_units, input_shape=(len_sequence, input_dim), activation = "relu", return_sequences=True)) # 2層目
        NN_model.add(SimpleRNN(NN_hidden_units, input_shape=(len_sequence, input_dim), activation = "relu", return_sequences=True)) # 3層目
        NN_model.add(SimpleRNN(NN_hidden_units, input_shape=(len_sequence, input_dim), activation = "relu", return_sequences=True)) # 4層目
        NN_model.add(Dense(output_dim))
        NN_model.compile(optimizer=Adam(lr=learning_rate), loss="mean_squared_error")

        ###### 学習 ######
        NN_model.fit(X_train, y_train, batch_size=batch_size, epochs=num_of_training_epochs, validation_split=0.1, verbose=0)

#################################### テストデータの予測と予測誤差の算出 #######################################
        ###### テストデータの予測
        testPredict_NN = NN_model.predict(X_test)
        
        ###### "テストデータの予測"から予測誤差を求める
        for day in range(0,test_days): # データを日ごとに分ける
            day_NN = testPredict_NN[day:(day + 1)]
            day_target = y_test[day:(day + 1)]
            err_day = (day_target - day_NN ) # 予測誤差の算出
            err_day = np.reshape(err_day, (48,1)) # 3次元から2次元に変換
            err_day = pd.DataFrame(err_day) 
            if day == 0:
                err = err_day
            elif day != 0:
                err = pd.concat([err,err_day], axis=1) # 30日分の誤差をまとめる(48×30)
                
####################################### 予測データの予測・結果の保存 #########################################
        ###### 具体的な予測データの日を指定する
        for days in range(0,pred_days):
            pred_data = PV_data[48*(train_days + test_days + days):48*(train_days + test_days + days + 1)] # 予測データ
            
            ###### 予測入力データ(X_pred)と正解データ(y_true)を用意する
            X_pred = pred_data[["sin_month","cos_month","sin_hour", "cos_hour","humidity","temp", "cloudcover", "rain"]]
            y_true = pred_data["generation"] 
            X_pred = X_pred.values.reshape((1, 48 ,8)) #型変換し三次元に変換
            #y_true = y_true.values
            y_true = y_true.values.reshape((1, 48, 1)) #型変換し三次元に変換
            
            #time_data = pred_data[["year","month","day","hour"]].reset_index(drop=True)
            #true_data = pred_data["generation"].reset_index(drop=True)

            ###### 予測データの予測 
            predPredict_NN = NN_model.predict(X_pred)
            predPredict_NN = np.reshape(predPredict_NN, (48,1)) # 3次元から2次元に変換

            ###### テストデータから得た"各時刻における予測誤差"と予測データから得た"予測発電出力" 
            ###### から各時刻における予測発電出力の上限値・予測下限値・Prediction Intervalを求める 
            for time in range(0,48):
                time_err = err.iloc[time] # 時刻別にデータを抽出
                time_err = sorted(time_err) # 昇順に並び替え
                time_err = pd.DataFrame(time_err).T
                time_err_low = time_err[round(pred_days*(1-0.95))] # ある時刻の予測誤差の下限値
                time_err_up = time_err[round(pred_days*0.95)] # ある時刻の予測誤差の上限値
                lower = predPredict_NN[time] + time_err_low # 予測発電出力の下限値
                upper = predPredict_NN[time] + time_err_up # 予測発電出力の上限値
                preint = upper - lower
                lower = pd.DataFrame(lower)
                upper = pd.DataFrame(upper)
                preint = pd.DataFrame(preint)
                if time == 0:
                    Lower = lower
                    Upper = upper
                    PreInt = preint
                elif time != 0:
                    Lower = pd.concat([Lower,lower], axis=1)
                    Upper = pd.concat([Upper,upper], axis=1)
                    PreInt = pd.concat([PreInt,preint], axis=1)

            Lower = (Lower.T).values
            Upper = (Upper.T).values
            PreInt = (PreInt.T).values

            ###### 予測日のグラフの表示
            y_true = np.reshape(y_true,(48, 1))
            pdfname = 'PV_result-'+str(days)+'.pdf'
            pp = PdfPages(pdfname) # PDFの作成
            graph_NN = graph(time_stamp, predPredict_NN, y_true, Lower, Upper, "NN", 3) # NN
            pp.savefig(graph_NN)
            pp.close()

            ###### 予測日のRMSE
            daytime_y_true = y_true[13:36]
            daytime_y_pred = predPredict_NN[13:36]
            NN_RMSE = RMSE(daytime_y_true,daytime_y_pred, "NN") # NN
            
            ###### 予測日のMPIWcapt,Cover Rate
            count = 0
            PIcapt = []
            for i in range(13,36):
                if Lower[i] <= y_true[i] <= Upper[i]:
                    PIcapt.append(PreInt[i])
                    count = count + 1
                else:
                    PIcapt.append(0)
            PI_cover_rate = count/23*100
            PIcapt_np = np.array(PIcapt,dtype = 'float64')
            PIcaptave = np.sum(PIcapt_np)/count

            ###### 予測日のLoss
            param = 496.0807 #Lossのハイパーパラメータ
            target_cover_rate = 95 #Cover Rateの目標値[%]
            if PI_cover_rate < target_cover_rate:
                Loss = PIcaptave + param * (((target_cover_rate - PI_cover_rate)/100) ** 2)
            else:
                Loss = PIcaptave

            ###### 結果の保存
            time_data = pred_data[["year","month","day","hour"]].reset_index(drop=True)
            true_data = pred_data["generation"].reset_index(drop=True)
            predPredict_NN = pd.DataFrame(predPredict_NN)
            Lower = pd.DataFrame(Lower).reset_index(drop=True)
            Upper = pd.DataFrame(Upper).reset_index(drop=True)
            PreInt = pd.DataFrame(PreInt).reset_index(drop=True)
            result = pd.concat([time_data,true_data,predPredict_NN,Lower,Upper,PreInt],axis=1)

            time_data = time_data.astype('float64')
            eva_result = []
            eva_result.append(NN_RMSE)
            eva_result.append(PI_cover_rate)
            eva_result.append(PIcaptave)
            eva_result.append(Loss)

            if days == 0:
                all_result = result
                all_eva_result = eva_result
            elif days != 0:
                all_result = pd.concat([all_result,result], axis=0)
                all_eva_result = np.vstack((all_eva_result,eva_result))

        all_eva_result = pd.DataFrame(all_eva_result)

        err_name = ["NN_RMSE","PI_cover_rate","PIcapt","Loss"] # 列名
        label_name = ["year","month","day","hour","PVout(true)","Forecast","Lower","Upper","PreInt"] # 列名

        all_eva_result.columns = err_name # 列名付与
        all_eva_result_savename = f'PV_predict_err_{method[m]}{missing_rate[mr]}.csv'
        all_eva_result.to_csv(all_eva_result_savename)
        save_file_at_dir(all_eva_result_savename,path_name)
        #all_eva_result.to_csv(all_eva_result_savename)

        all_result.columns = label_name # 列名付与
        all_result_savename = f'PV_predict_result_{method[m]}{missing_rate[mr]}.csv'
        all_result.to_csv(all_result_savename)
        #save_file_at_dir(all_result_savename,path_name)

        ###### pdfをまとめてフォルダに移動
        for days in range(31):
            pdfname = 'PV_result-'+str(days)+'.pdf'
            save_file_at_dir(pdfname,path_name)

C:\Users\Yu\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1/1 [==============================] - 0s 14ms/step


<ipython-input-4-63e5f994ef02>:152: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  PIcapt_np = np.array(PIcapt,dtype = 'float64')


1/1 [==============================] - 0s 15ms/step


<ipython-input-4-63e5f994ef02>:152: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  PIcapt_np = np.array(PIcapt,dtype = 'float64')


...

<ipython-input-4-63e5f994ef02>:152: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  PIcapt_np = np.array(PIcapt,dtype = 'float64')


1/1 [==============================] - 0s 13ms/step


<ipython-input-4-63e5f994ef02>:152: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  PIcapt_np = np.array(PIcapt,dtype = 'float64')


1/1 [==============================] - 0s 14ms/step


Error: Destination path './Mean20%\PV_predict_err_Mean20.csv' already exists